In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drug-classification/drug200.csv


In [140]:
df=pd.read_csv("/kaggle/input/drug-classification/drug200.csv")

In [142]:
df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [144]:
drug_variable=['Sex','BP','Cholesterol','Drug']

In [145]:
df=pd.get_dummies(data=df,prefix=drug_variable,columns=drug_variable)

In [146]:
df

,Age,Na_to_K,Sex_F,Sex_M,BP_HIGH,BP_LOW,BP_NORMAL,Cholesterol_HIGH,Cholesterol_NORMAL,Drug_DrugY,Drug_drugA,Drug_drugB,Drug_drugC,Drug_drugX
0,23,25.355,True,False,True,False,False,True,False,True,False,False,False,False
1,47,13.093,False,True,False,True,False,True,False,False,False,False,True,False
2,47,10.114,False,True,False,True,False,True,False,False,False,False,True,False
3,28,7.798,True,False,False,False,True,True,False,False,False,False,False,True
4,61,18.043,True,False,False,True,False,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,56,11.567,True,False,False,True,False,True,False,False,False,False,True,False
196,16,12.006,False,True,False,True,False,True,False,False,False,False,True,False
197,52,9.894,False,True,False,False,True,True,False,False,False,False,False,True
198,23,14.020,False,True,False,False,True,False,True,False,False,False,False,True


In [156]:
df.replace({True: 1, False: 0})

,Age,Na_to_K,Sex_F,Sex_M,BP_HIGH,BP_LOW,BP_NORMAL,Cholesterol_HIGH,Cholesterol_NORMAL,Drug_DrugY,Drug_drugA,Drug_drugB,Drug_drugC,Drug_drugX
0,23,25.355,1,0,1,0,0,1,0,1,0,0,0,0
1,47,13.093,0,1,0,1,0,1,0,0,0,0,1,0
2,47,10.114,0,1,0,1,0,1,0,0,0,0,1,0
3,28,7.798,1,0,0,0,1,1,0,0,0,0,0,1
4,61,18.043,1,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,56,11.567,1,0,0,1,0,1,0,0,0,0,1,0
196,16,12.006,0,1,0,1,0,1,0,0,0,0,1,0
197,52,9.894,0,1,0,0,1,1,0,0,0,0,0,1
198,23,14.020,0,1,0,0,1,0,1,0,0,0,0,1


In [157]:
features = [x for x in df.columns if x not in ('Drug_DrugY','Drug_drugA','Drug_drugB','Drug_drugC','Drug_drugX')] 
output_features=df[['Drug_DrugY','Drug_drugA','Drug_drugB','Drug_drugC','Drug_drugX']]

In [158]:
features

['Age',
 'Na_to_K',
 'Sex_F',
 'Sex_M',
 'BP_HIGH',
 'BP_LOW',
 'BP_NORMAL',
 'Cholesterol_HIGH',
 'Cholesterol_NORMAL']

In [159]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV

from xgboost import XGBClassifier

In [160]:
X_train,X_test,Y_train,Y_test=train_test_split(df[features],output_features,test_size=0.3)

In [161]:
X_train

,Age,Na_to_K,Sex_F,Sex_M,BP_HIGH,BP_LOW,BP_NORMAL,Cholesterol_HIGH,Cholesterol_NORMAL
113,65,13.769,1,0,0,1,0,0,1
5,22,8.607,1,0,0,0,1,1,0
23,48,15.036,1,0,0,1,0,1,0
64,60,13.303,1,0,1,0,0,1,0
164,16,19.007,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
174,42,12.766,0,1,1,0,0,0,1
187,47,10.403,0,1,1,0,0,1,0
119,61,25.475,1,0,1,0,0,1,0
21,63,25.917,0,1,0,0,1,1,0


In [162]:
Y_train

,Drug_DrugY,Drug_drugA,Drug_drugB,Drug_drugC,Drug_drugX
113,0,0,0,0,1
5,0,0,0,0,1
23,1,0,0,0,0
64,0,0,1,0,0
164,1,0,0,0,0
...,...,...,...,...,...
174,0,1,0,0,0
187,0,1,0,0,0
119,1,0,0,0,0
21,1,0,0,0,0


In [168]:
param_grid = {
    'n_estimators': [50, 100, 200,300],  # You can adjust these values
    'min_samples_split': [2,3,5,10],
    'max_depth': [None, 10, 20, 30]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, Y_train)
print("Best parameters found: ", grid_search.best_params_)


Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best parameters found:  {'max_depth': None, 'min_samples_split': 3, 'n_estimators': 300}


In [169]:
model=RandomForestClassifier(n_estimators=300,min_samples_split=3)
model.fit(X_train,Y_train)
predictions_train=model.predict(X_train)
predictions_test=model.predict(X_test)
accuracy_train = accuracy_score(predictions_train,Y_train)
accuracy_test = accuracy_score(predictions_test,Y_test)


In [170]:
accuracy_train*100,accuracy_test*100

(100.0, 96.66666666666667)

In [174]:
drug_labels = ['Y', 'A', 'B', 'C', 'X']
def convert_to_labels(predictions):
    return [drug_labels[np.argmax(pred)] for pred in predictions]
predicted_drugs_train = convert_to_labels(predictions_train)
predicted_drugs_test=convert_to_labels(predictions_test)

In [175]:
predicted_drugs_train

['X',
 'X',
 'Y',
 'B',
 'Y',
 'Y',
 'A',
 'Y',
 'X',
 'B',
 'Y',
 'Y',
 'X',
 'Y',
 'Y',
 'C',
 'Y',
 'Y',
 'Y',
 'Y',
 'Y',
 'Y',
 'A',
 'Y',
 'Y',
 'X',
 'B',
 'Y',
 'Y',
 'X',
 'C',
 'X',
 'X',
 'Y',
 'A',
 'Y',
 'X',
 'X',
 'Y',
 'B',
 'Y',
 'Y',
 'A',
 'A',
 'Y',
 'X',
 'Y',
 'X',
 'X',
 'X',
 'X',
 'X',
 'Y',
 'B',
 'Y',
 'Y',
 'B',
 'Y',
 'Y',
 'Y',
 'Y',
 'Y',
 'Y',
 'Y',
 'B',
 'X',
 'A',
 'Y',
 'X',
 'Y',
 'X',
 'C',
 'X',
 'Y',
 'C',
 'Y',
 'C',
 'X',
 'B',
 'Y',
 'C',
 'A',
 'C',
 'A',
 'X',
 'X',
 'Y',
 'Y',
 'X',
 'Y',
 'Y',
 'Y',
 'A',
 'A',
 'Y',
 'Y',
 'X',
 'Y',
 'Y',
 'X',
 'Y',
 'X',
 'A',
 'Y',
 'X',
 'Y',
 'Y',
 'A',
 'X',
 'Y',
 'C',
 'X',
 'X',
 'C',
 'X',
 'Y',
 'B',
 'X',
 'A',
 'Y',
 'X',
 'X',
 'A',
 'X',
 'Y',
 'A',
 'Y',
 'Y',
 'Y',
 'X',
 'X',
 'Y',
 'Y',
 'Y',
 'C',
 'A',
 'A',
 'Y',
 'Y',
 'Y']